In [ ]:
# import libraries
import serial
import pandas as pd
from pandas import DataFrame
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import threading
import os

# configure Matplotlib backend
import matplotlib
matplotlib.use('TkAgg')

# functions

# reads in the pressure data from the sensor and writes pressure and time data to csv
def pressure_read(serial_port, num_readings, filename, samplerate, done_flag):
    ser = serial.Serial(serial_port, 115200)
    data = []

    for _ in range(num_readings):
        line = ser.readline().decode().rstrip()
        data.append(float(line))

        df = DataFrame(data, columns=['pressure'])
        df['real_time'] = np.arange(len(data)) * samplerate
        df.to_csv(f'{filename}.csv', index=False)
    
    ser.close()
    done_flag[0] = False

# live plots pressure vs time 
def update_plot(i, filename, done_flag):
    csv_file = f'{filename}.csv'
    if os.path.exists(csv_file) and os.path.getsize(csv_file) > 0:
        df = pd.read_csv(csv_file)
        plt.cla()
        plt.plot(df['real_time'], df['pressure'])
        plt.xlabel('Time (seconds)')
        plt.ylabel('Pressure Reading')
        plt.title('Pressure vs. Time')
    
    # Check if the data collection is done and save the plot
    if not done_flag[0]:
        plt.savefig(f'{filename}.png')
        plt.close()

# runs the code
def main(number_pressure_readings,samplerate, serial_port, filename):

    done_flag = [True]

    thread = threading.Thread(target=pressure_read, args=(serial_port, number_pressure_readings, filename, samplerate, done_flag))
    thread.start()

    animation = FuncAnimation(plt.gcf(), lambda i: update_plot(i, filename, done_flag), 
                              interval=100, cache_frame_data=False)
    plt.show()

    done_flag[0] = False
    thread.join()

In [ ]:
# specify parameters
number_pressure_readings = 100
samplerate = 0.1
serial_port = '/dev/cu.usbmodem2101'
filename = 'test'

In [ ]:
# run pressure reading, writing to csv, and plotting
if __name__ == "__main__":
    main(number_pressure_readings,samplerate, serial_port, filename)